In [1]:
# 基本ライブラリ
import pandas as pd
import pandas.io.sql as psql
import numpy as np
import numpy.random as rd
import gc
import multiprocessing as mp
import os
import sys
import pickle
from collections import defaultdict
from glob import glob
import math
from datetime import datetime as dt
from pathlib import Path
import scipy.stats as st
import re
import shutil
from tqdm import tqdm_notebook as tqdm
import datetime
ts_conv = np.vectorize(datetime.datetime.fromtimestamp) # 秒ut(10桁) ⇒ 日付

# グラフ描画系
import matplotlib
from matplotlib import font_manager
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from matplotlib import rc

from matplotlib import animation as ani
from IPython.display import Image

plt.rcParams["patch.force_edgecolor"] = True
#rc('text', usetex=True)
from IPython.display import display # Allows the use of display() for DataFrames
import seaborn as sns
sns.set(style="whitegrid", palette="muted", color_codes=True)
sns.set_style("whitegrid", {'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

#カラム内の文字数。デフォルトは50
pd.set_option("display.max_colwidth", 100)

#行数
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
#
pd.options.display.float_format = '{:,.5f}'.format

%matplotlib inline

In [2]:
import lightgbm as lgb
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold

from sklearn import metrics
import json

import warnings
warnings.filterwarnings("ignore")


sys.path.append('..')
from lib.line_notif import send_message
from lib.utils import reduce_mem_usage, current_time, unpickle, to_pickle
from lib.utils import one_hot_encoder, apply_agg, multi_combine_categorical_feature
from lib.utils import import_data, get_split_indexer 

In [3]:
sub = pd.read_csv('../input/sample_submission.csv')

In [4]:
DATA_VERSION = "v001"
TRIAL_NO = "005"
save_path = Path(f"../processed/{DATA_VERSION}")
model_path = Path(f"../model/{DATA_VERSION}_{TRIAL_NO}")
submit_path = Path(f"../submit/{DATA_VERSION}_{TRIAL_NO}")

In [5]:
print("start loading...")
test  = unpickle(save_path/"test_002.df.pkl", )
print(f"test loaded.")
test.set_index("id", inplace=True)
print(f"test: {test.shape}")
print(test.shape,)
groups = unpickle(save_path/"lbl_molecule_name.pkl", )


def get_test_data_for_predict(mol_type):
    # mol_type = 1 # int(sys.argv[1]) #1
    print(f"mol_type: {mol_type}")

    if mol_type==0:
        pass
    elif mol_type==1:
        test_type_cut = np.load("../processed/v001/test_type1_cut.npy", )
    elif mol_type==2:
        test_type_cut = np.load("../processed/v001/test_type2_cut.npy", )
    elif mol_type==3:
        test_type_cut = np.load("../processed/v001/test_type3_cut.npy", )
    else:
        assert False, f"mol_type should be 0, 1, 2, 3. mol_type: {mol_type}"

#     if mol_type in [1,2,3]:
#         test_  = test[test_type_cut]
    return test[test_type_cut]

def get_train_data_for_predict(mol_type):
    # mol_type = 1 # int(sys.argv[1]) #1
    print(f"mol_type: {mol_type}")

    if mol_type==0:
        pass
    elif mol_type==1:
        train_type_cut = np.load("../processed/v001/train_type1_cut.npy", )
    elif mol_type==2:
        train_type_cut = np.load("../processed/v001/train_type2_cut.npy", )
    elif mol_type==3:
        train_type_cut = np.load("../processed/v001/train_type3_cut.npy", )
    else:
        assert False, f"mol_type should be 0, 1, 2, 3. mol_type: {mol_type}"

    return train[train_type_cut]

start loading...
test loaded.
test: (2505542, 448)
(2505542, 448)


In [ ]:
sub_list = []
n_folds = 5
iterations = [29000, 7000, 20000]

for mol_type, iteration in zip([1,2,3], iterations):
    print(f"mol_type: {mol_type}")
    test_ = get_test_data_for_predict(mol_type=mol_type)
    #bst_list = []
    preds_test = np.zeros((test_.shape[0], n_folds))
    for fold_ in tqdm(range(n_folds)):
        saved_model_path = f'../model/{DATA_VERSION}_{TRIAL_NO}/booster_{mol_type}_{fold_:02d}_{iteration}.model'
        bst = lgb.Booster(model_file=saved_model_path)
        #bst_list += [bst]
        preds_test[:, fold_] = bst.predict(test_)

    sub = pd.DataFrame(index=test_.index)
    sub["scalar_coupling_constant"] = preds_test.mean(axis=1)    
    sub_list.append(sub)
    
sub_all = pd.concat(sub_list, axis=0)    
sub_all.to_csv(submit_path/f"sub_all_{DATA_VERSION}_{TRIAL_NO}.csv")

In [15]:
!head ../submit/v001_005/sub_all.csv

id,scalar_coupling_constant
4658148,162.6267045798958
4658150,150.5427402349208
4658152,91.57319688567192
4658156,92.30873384342375
4658159,82.59355472025568
4658162,90.01716055160892
4658166,90.0016414843507
4658169,82.50557919822882
4658170,110.9599438151333


In [16]:
# !cp ../submit/v001_005/sub_all.csv ../submit/v001_005/sub_all_v001_005.csv

In [18]:
# !kaggle competitions submit -c champs-scalar-coupling -f ../submit/v001_005/sub_all_v001_005.csv -m "first submit"

100%|██████████████████████████████████████| 64.0M/64.0M [00:04<00:00, 14.8MB/s]
Successfully submitted to Predicting Molecular Properties

In [9]:
n_folds = 5
iterations = [29000, 7000, 20000]

importance_list = []
for mol_type, iteration in zip([1,2,3], iterations):
    print(f"mol_type: {mol_type}")
    importance_sub = []
    for fold_ in tqdm(range(n_folds)):
        saved_model_path = f'../model/{DATA_VERSION}_{TRIAL_NO}/booster_{mol_type}_{fold_:02d}_{iteration}.model'
        bst = lgb.Booster(model_file=saved_model_path)
        importance_sub += [pd.DataFrame({"gain":bst.feature_importance(importance_type='gain')},
                                       index=test.columns).sort_index()]
    importance_df = pd.concat(importance_sub, axis=1)
    importance_df.columns = [f"fold_{i}" for i in range(1,6)]
    importance_df["ave"] = importance_df.mean(axis=1)
    importance_df["ratio"] = importance_df["ave"]/importance_df["ave"].sum()
    importance_list += [importance_df.sort_values("ave", ascending=False)]

mol_type: 1



mol_type: 2



mol_type: 3


In [ ]:
for i in range(3):
    df_tmp = importance_list[i][importance_list[i].ratio>=0.0001]
    print(importance_list[i].shape, df_tmp.shape)
    np.save(f"../model/{DATA_VERSION}_{TRIAL_NO}/top_importance_{i+1}.npy", np.array(df_tmp.index))
    #display(df_tmp)

In [10]:
for i in range(3):
    df_tmp = importance_list[i][importance_list[i].ratio<0.0001]
    print(importance_list[i].shape, df_tmp.shape)
    np.save(f"../model/{DATA_VERSION}_{TRIAL_NO}/low_importance_{i+1}.npy", np.array(df_tmp.index))
    #display(df_tmp)

(448, 7) (254, 7)
(448, 7) (302, 7)
(448, 7) (137, 7)


In [17]:
ls ../submit/v001_005

sub_all.csv  sub_all_v001_005.csv


# calc oof score

In [6]:
print("start loading...")
train  = unpickle(save_path/"train_002.df.pkl", )
print(f"train loaded.")
train.set_index("id", inplace=True)
print(f"train: {train.shape}")

start loading...
train loaded.
train: (4658147, 449)


In [7]:
y = train["scalar_coupling_constant"]
train.drop("scalar_coupling_constant", axis=1, inplace=True)

In [13]:
train["type"]

,atom_index_0,atom_index_1,type,atom_0,x_0,y_0,z_0,atom_1,x_1,y_1,z_1,dist,dist_x,dist_y,dist_z,type_0,molecule_couples,molecule_dist_mean,molecule_dist_min,molecule_dist_max,molecule_dist_std,atom_0_couples_count,atom_1_couples_count,molecule_atom_index_0_count,molecule_atom_index_1_count,molecule_type_count,molecule_atom_1_count,molecule_type_0_count,molecule_atom_index_0_x_1_mean,molecule_atom_index_0_x_1_mean_diff,molecule_atom_index_0_x_1_mean_div,molecule_atom_index_0_x_1_max,molecule_atom_index_0_x_1_max_diff,molecule_atom_index_0_x_1_max_div,molecule_atom_index_0_x_1_std,molecule_atom_index_0_x_1_std_diff,molecule_atom_index_0_x_1_std_div,molecule_atom_index_0_x_1_min,molecule_atom_index_0_x_1_min_diff,molecule_atom_index_0_x_1_min_div,molecule_atom_index_0_y_1_mean,molecule_atom_index_0_y_1_mean_diff,molecule_atom_index_0_y_1_mean_div,molecule_atom_index_0_y_1_max,molecule_atom_index_0_y_1_max_diff,molecule_atom_index_0_y_1_max_div,molecule_atom_index_0_y_1_std,molecule_atom_index_0_y_1_std_diff,molecule_atom_index_0_y_1_std_div,molecule_atom_index_0_y_1_min,molecule_atom_index_0_y_1_min_diff,molecule_atom_index_0_y_1_min_div,molecule_atom_index_0_z_1_mean,molecule_atom_index_0_z_1_mean_diff,molecule_atom_index_0_z_1_mean_div,molecule_atom_index_0_z_1_max,molecule_atom_index_0_z_1_max_diff,molecule_atom_index_0_z_1_max_div,molecule_atom_index_0_z_1_std,molecule_atom_index_0_z_1_std_diff,molecule_atom_index_0_z_1_std_div,molecule_atom_index_0_z_1_min,molecule_atom_index_0_z_1_min_diff,molecule_atom_index_0_z_1_min_div,molecule_atom_index_0_dist_mean,molecule_atom_index_0_dist_mean_diff,molecule_atom_index_0_dist_mean_div,molecule_atom_index_0_dist_max,molecule_atom_index_0_dist_max_diff,molecule_atom_index_0_dist_max_div,molecule_atom_index_0_dist_std,molecule_atom_index_0_dist_std_diff,molecule_atom_index_0_dist_std_div,molecule_atom_index_0_dist_min,molecule_atom_index_0_dist_min_diff,molecule_atom_index_0_dist_min_div,molecule_atom_index_0_dist_x_mean,molecule_atom_index_0_dist_x_mean_diff,molecule_atom_index_0_dist_x_mean_div,molecule_atom_index_0_dist_x_max,molecule_atom_index_0_dist_x_max_diff,molecule_atom_index_0_dist_x_max_div,molecule_atom_index_0_dist_x_std,molecule_atom_index_0_dist_x_std_diff,molecule_atom_index_0_dist_x_std_div,molecule_atom_index_0_dist_x_min,molecule_atom_index_0_dist_x_min_diff,molecule_atom_index_0_dist_x_min_div,molecule_atom_index_0_dist_y_mean,molecule_atom_index_0_dist_y_mean_diff,molecule_atom_index_0_dist_y_mean_div,molecule_atom_index_0_dist_y_max,molecule_atom_index_0_dist_y_max_diff,molecule_atom_index_0_dist_y_max_div,molecule_atom_index_0_dist_y_std,molecule_atom_index_0_dist_y_std_diff,molecule_atom_index_0_dist_y_std_div,molecule_atom_index_0_dist_y_min,molecule_atom_index_0_dist_y_min_diff,molecule_atom_index_0_dist_y_min_div,molecule_atom_index_0_dist_z_mean,molecule_atom_index_0_dist_z_mean_diff,molecule_atom_index_0_dist_z_mean_div,molecule_atom_index_0_dist_z_max,molecule_atom_index_0_dist_z_max_diff,molecule_atom_index_0_dist_z_max_div,molecule_atom_index_0_dist_z_std,molecule_atom_index_0_dist_z_std_diff,molecule_atom_index_0_dist_z_std_div,molecule_atom_index_0_dist_z_min,molecule_atom_index_0_dist_z_min_diff,molecule_atom_index_0_dist_z_min_div,molecule_atom_index_1_x_1_mean,molecule_atom_index_1_x_1_mean_diff,molecule_atom_index_1_x_1_mean_div,molecule_atom_index_1_x_1_max,molecule_atom_index_1_x_1_max_diff,molecule_atom_index_1_x_1_max_div,molecule_atom_index_1_x_1_std,molecule_atom_index_1_x_1_std_diff,molecule_atom_index_1_x_1_std_div,molecule_atom_index_1_x_1_min,molecule_atom_index_1_x_1_min_diff,molecule_atom_index_1_x_1_min_div,molecule_atom_index_1_y_1_mean,molecule_atom_index_1_y_1_mean_diff,molecule_atom_index_1_y_1_mean_div,molecule_atom_index_1_y_1_max,molecule_atom_index_1_y_1_max_diff,molecule_atom_index_1_y_1_max_div,molecule_atom_index_1_y_1_std,molecule_atom_index_1_y_1_std_diff,molecule_atom_index_1_y_1_std_div,molecule_atom_index_1_y_1_min,mol

In [11]:
train_base = pd.read_csv('../input/train.csv')

In [12]:
train_base.head()

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant
0,0,dsgdb9nsd_000001,1,0,1JHC,84.80760
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.25700
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.25480
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.25430
4,4,dsgdb9nsd_000001,2,0,1JHC,84.80740


In [ ]:
oof_list = []
n_folds = 5
iterations = [29000, 7000, 20000]
train_idx = []
for mol_type, iteration in zip([1,2,3], iterations):
    print(f"mol_type: {mol_type}")
    train_ = get_train_data_for_predict(mol_type=mol_type)
    #bst_list = []
    train_idx.extend(train_.index.tolist())
    preds_oof = np.zeros((train_.shape[0], n_folds))
    for fold_ in tqdm(range(n_folds)):
        saved_model_path = f'../model/{DATA_VERSION}_{TRIAL_NO}/booster_{mol_type}_{fold_:02d}_{iteration}.model'
        bst = lgb.Booster(model_file=saved_model_path)
        #bst_list += [bst]
        preds_oof[:, fold_] = bst.predict(train_)

    oof = pd.DataFrame(index=train_.index)
    oof["scalar_coupling_constant"] = preds_oof.mean(axis=1)    
    oof_list.append(oof)
    
oof_all = pd.concat(oof_list, axis=0)    


mol_type: 1
mol_type: 1


In [ ]:

def group_mean_log_mae(y_true, y_pred, types, floor=1e-9):
    """
    Fast metric computation for this competition: https://www.kaggle.com/c/champs-scalar-coupling
    Code is from this kernel: https://www.kaggle.com/uberkinder/efficient-metric
    """
    maes = (y_true-y_pred).abs().groupby(types).mean()
    return np.log(maes.map(lambda x: max(x, floor))).mean()

In [ ]:
score = group_mean_log_mae(y, oof_all[train_idx], train["type"])

In [ ]:
print(f"group_mean_log_mae: {score:.5f}")